# Data Preparation 

In [85]:
# load packages

import numpy as np
import pandas as pd

In [33]:
# load dataset 

Chicago_Crimes_2005_to_2007 = pd.read_csv("crimes-in-chicago/Chicago_Crimes_2005_to_2007.csv", usecols = range(23))
Chicago_Crimes_2008_to_2011 = pd.read_csv("crimes-in-chicago/Chicago_Crimes_2008_to_2011.csv", usecols = range(23))
Chicago_Crimes_2012_to_2017 = pd.read_csv("crimes-in-chicago/Chicago_Crimes_2012_to_2017.csv", usecols = range(23))

# merge data 
Chicago_Crimes_2005_to_2017 = pd.concat([Chicago_Crimes_2005_to_2007, Chicago_Crimes_2008_to_2011, Chicago_Crimes_2012_to_2017])

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
# filter the dataset by years and export data from 2007 to 2017

Chicago_Crimes_2005_to_2017.head(20)
print(Chicago_Crimes_2005_to_2017.Year.unique())
Chicago_Crimes_2007_to_2017 = Chicago_Crimes_2005_to_2017.loc[(Chicago_Crimes_2005_to_2017['Year'] >= 2007) 
                                                              & (Chicago_Crimes_2005_to_2017['Year'] <= 2017)]
print(Chicago_Crimes_2007_to_2017.Year.unique())

# store the new DataFrame to a csv file for later use.
Chicago_Crimes_2007_to_2017.to_csv("Chicago_Crimes_2007_to_2017.csv",index=False)

[2006 2005 2007 2008 2009 2011 2010 2016 2015 2012 2014 2013 2017]
[2007 2008 2009 2011 2010 2016 2015 2012 2014 2013 2017]


In [64]:
# load data from 2007 to 2017
df = pd.read_csv("Chicago_Crimes_2007_to_2017.csv")

# check for features 
print(df.columns.values)

# drop unnecessary features, col 0
df = df.drop(df.columns[0], axis=1)
print(df.dtypes)
print(df.shape) # (4496569, 22), 22 features 

# removing columns X/Y Coordinate, Updated On, Latitude, Longitude
df.drop(['X Coordinate', 'Y Coordinate', 'Updated On', 'Latitude', 'Longitude'], axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['Unnamed: 0' 'ID' 'Case Number' 'Date' 'Block' 'IUCR' 'Primary Type'
 'Description' 'Location Description' 'Arrest' 'Domestic' 'Beat'
 'District' 'Ward' 'Community Area' 'FBI Code' 'X Coordinate'
 'Y Coordinate' 'Year' 'Updated On' 'Latitude' 'Longitude' 'Location']
ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude                object
Location                 object
dtype: object
(4496569, 22)


In [65]:
# check uniqueness of ID

df['ID'].is_unique #no unique, cases being recorded multiple times 

False

In [66]:
# return only one record per case by removing duplicated rows 
# ex: df[df['ID'] == 7781132]; 7815319, 7781132, 6023200  

print(len(df.ID.unique())) #3182854, but we have 4496569 rows 
df.drop_duplicates(keep = 'first', inplace = True) #remove duplicates 

# drop Case number, we are using ID as the primary key
df.drop(['Case Number'], axis = 1, inplace = True)
df.reset_index(inplace = True)

print(df.shape) #(3182854, 16) #matched 

3182854
(3182854, 16)


In [74]:
# check for missing values

df.isna().any() # Case Number, Location Description, District, Ward, Community Area

# Replace null values with 'Other' option; categorical 
df['Location Description'].value_counts() # 154 types + other
df['Location Description'].isna().sum() # 1933 null values 
df['Location Description'].fillna("Other", inplace = True)

# Numerical missing values 
df['District'].value_counts() # 25districts + nan
df['District'].isna().sum() # 43 nan

df['Ward'].value_counts() # 50 + nan
df['Ward'].isna().sum() # 56 nan

df['Community Area'].value_counts() # 78 + nan
df['Community Area'].isna().sum() # 926 nan

############TBC

926

In [104]:
# separate Date column into two col: date and time 

df.Date.head(10)
datetime = pd.to_datetime(df.Date, errors='coerce')

df.drop(['Date'], axis = 1, inplace = True)
df['Date'] = datetime.dt.date
df['Time'] = datetime.dt.time

df.dtypes

index                     int64
ID                        int64
Case Number              object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
Year                      int64
Date                     object
Time                     object
dtype: object

In [ ]:
df.to_csv("cleanedData_ver1.csv",index=False)